<a href="https://colab.research.google.com/github/malowana/NLP_budzet_obywatelski/blob/main/BO_analiza_morfologiczna_LDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyLDAvis

In [115]:
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', 'SelectableGroups dict interface')

In [116]:
import pandas as pd
import numpy as np
import gensim
from gensim import corpora, models
from gensim.utils import simple_preprocess
import pyLDAvis.gensim_models as vis
import pyLDAvis

In [117]:
# Wyświetlanie szerszych kolumn, aby nie skracać wyświetlanych nazw tematów
pd.set_option('max_colwidth', 400)

## Warszawa

In [118]:
df_waw = pd.read_csv('/content/drive/My Drive/Konkursy Kaggle/Budżet obywatelski/Morfeusz_WAW.csv')
df_waw.head()

,rok,miasto,nazwa,start_ind,end_ind,form,lemma,tag,qualifiers,_
0,2018,WARSZAWA,Wolskie lato filmowe,0,1,Wolskie,wolski,adj:pl:acc:m2.m3.f.n:pos,[],[]
1,2018,WARSZAWA,Wolskie lato filmowe,0,1,Wolskie,wolski,adj:pl:nom.voc:m2.m3.f.n:pos,[],[]
2,2018,WARSZAWA,Wolskie lato filmowe,0,1,Wolskie,wolski,adj:sg:acc:n:pos,[],[]
3,2018,WARSZAWA,Wolskie lato filmowe,0,1,Wolskie,wolski,adj:sg:nom.voc:n:pos,[],[]
4,2018,WARSZAWA,Wolskie lato filmowe,0,1,Wolskie,Wolskie,subst:pl:nom.acc.voc:n:pt,['nazwa_geograficzna'],[]


In [119]:
df_waw['lemma_clean'] = df_waw['lemma'].str.split(':', expand=True)[0]
df_waw.sample(5)

,rok,miasto,nazwa,start_ind,end_ind,form,lemma,tag,qualifiers,_,lemma_clean
25556,2018,WARSZAWA,"zdrowie na Służewcu: joga, fitness, tai chi w plenerze, za darmo, dla każdego",14,15,darmo,darmo,adv,[],[],darmo
23325,2018,WARSZAWA,"Mini Studio Audiowizualne w Domu Kultury ""Wygoda""",0,1,Mini,mini:A,adj:sg:nom.voc:f:pos,[],[],mini
33995,2019,WARSZAWA,"Rewitalizacja placu zabaw , boiska oraz mini skateparku wraz z miasteczkiem ruchu drogowego przy pętli Gocławek",6,7,mini,mini:A,adj:pl:dat:m1.m2.m3.f.n:pos,[],[],mini
20401,2018,WARSZAWA,„Bądźmy kreatywni” – cykl bezpłatnych zajęć plastycznych dla dzieci i dorosłych,6,7,bezpłatnych,bezpłatny,adj:pl:gen:m1.m2.m3.f.n:pos,[],[],bezpłatny
74816,2020,WARSZAWA,Modernizacja sali gimnastycznej Szkoły Podstawowej nr 258 filia Namysłowska,3,4,Szkoły,Szkoła:Sf,subst:sg:gen:f,['nazwisko'],[],Szkoła


In [120]:
df_waw['tokens'] = df_waw['lemma_clean'].apply(simple_preprocess)
df_waw.sample(5)

,rok,miasto,nazwa,start_ind,end_ind,form,lemma,tag,qualifiers,_,lemma_clean,tokens
20630,2018,WARSZAWA,"Pij, Warszawo - na Starej Ochocie",6,7,Ochocie,Ochot:Sm1,subst:sg:loc:m1,['nazwisko'],[],Ochot,[ochot]
36197,2019,WARSZAWA,"Usuwanie barier na rowerowym ""Słonecznym Szlaku"". Zniwelowanie wystającej komory telekomunikacyjnej",9,10,Zniwelowanie,zniwelować,ger:sg:nom.acc:n:perf:aff,[],[],zniwelować,[zniwelować]
19543,2018,WARSZAWA,Tęczowa Wałbrzyska - modernizacja Placu Zabaw przy ul. Wałbrzyskiej 48,9,10,Wałbrzyskiej,wałbrzyski,adj:sg:gen:f:pos,[],[],wałbrzyski,[wałbrzyski]
77980,2020,WARSZAWA,Integracyjne pikniki podwórkowe na Bródnie,3,4,na,na:P,prep:acc,[],[],na,[na]
17359,2018,WARSZAWA,Street workout- plac do ćwiczeń na drabinkach w parku stanisława dygata na sadybie,8,9,w,w,prep:acc:nwok,[],[],w,[]


In [121]:
df_waw3 = df_waw[df_waw["tokens"].str.len() != 0]
df_waw3.sample(10)

,rok,miasto,nazwa,start_ind,end_ind,form,lemma,tag,qualifiers,_,lemma_clean,tokens
80446,2020,WARSZAWA,Kurs pierwszej pomocy dla uczniów publicznych szkół podstawowych na Ursynowie,1,2,pierwszej,pierwszy:A,adj:sg:dat:f:pos,[],[],pierwszy,[pierwszy]
2063,2018,WARSZAWA,Chronimy jerzyki i wróble w Śródmieściu-skrzynki lęgowe,7,8,skrzynki,skrzynka,subst:sg:gen:f,['nazwa_pospolita'],[],skrzynka,[skrzynka]
69350,2020,WARSZAWA,Zielony Mokotów - nowe drzewa antysmogowe wzdłuż Alei Polski Walczącej,6,7,wzdłuż,wzdłuż:D,adv,[],[],wzdłuż,[wzdłuż]
20199,2018,WARSZAWA,"Miejsca postojowe przy Pustola 25, nowy chodnik oraz aranżacja zieleni",10,11,zieleni,zielony:A,adj:pl:nom.voc:m1:pos,[],[],zielony,[zielony]
49064,2019,WARSZAWA,Wygodne połączenie ciągu pieszego z ulicą Opieńki przy Parku Kultury w Powsinie,9,10,Kultury,kultura,subst:pl:nom.acc.voc:f,['nazwa_pospolita'],[],kultura,[kultura]
74769,2020,WARSZAWA,Zielone ściany i mniej betonu dla Pragi,0,1,Zielone,Zielona,subst:pl:nom.acc.voc:f,"['nazwa_geograficzna', 'nazwisko']",[],Zielona,[zielona]
25390,2018,WARSZAWA,Remont chodnika wzdłuż ulicy Suchodolskiej 24,3,4,ulicy,ulica,subst:sg:gen:f,['nazwa_pospolita'],[],ulica,[ulica]
7000,2018,WARSZAWA,Odkrywcy i marzyciele. Warsztaty naukowe i kreatywne dla dzieci,5,6,naukowe,naukowy,adj:sg:acc:n:pos,[],[],naukowy,[naukowy]
13087,2018,WARSZAWA,"Automat vendingowy, który nagrodzi nas za wrzucane do niego śmieci",3,4,który,który,adj:sg:nom.voc:m1.m2.m3:pos,[],[],który,[który]
37386,2019,WARSZAWA,Więcej zieleni mniej betonu na Wilanowie Wysokim,1,2,zieleni,zieleń,subst:pl:gen:f,['nazwa_pospolita'],[],zieleń,[zieleń]


In [122]:
processed_docs = df_waw3['tokens']

In [123]:
vocab = gensim.corpora.Dictionary(processed_docs)
len(vocab)

5958

In [124]:
bow_corpus = [vocab.doc2bow(doc) for doc in processed_docs]
bow_corpus[10]

[(4, 1)]

In [125]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=vocab, passes=2)

In [126]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))
    print("")

Topic: 0 
Words: 0.117*"wola" + 0.031*"plenerowy" + 0.027*"co" + 0.027*"praga" + 0.020*"spotkać" + 0.019*"dorosła" + 0.018*"oświetlenie" + 0.017*"miejsce" + 0.017*"polski" + 0.017*"radość"

Topic: 1 
Words: 0.068*"przy" + 0.037*"warsztaty" + 0.034*"rowerowy" + 0.020*"oświetlić" + 0.020*"numer" + 0.020*"świąteczny" + 0.019*"zielone" + 0.017*"wole" + 0.016*"pod" + 0.014*"rok"

Topic: 2 
Words: 0.044*"zająć" + 0.044*"zajęcie" + 0.030*"on" + 0.030*"kultura" + 0.029*"zabawa" + 0.024*"żoliborski" + 0.023*"edukacyjny" + 0.023*"zajęcia" + 0.018*"wesoły" + 0.018*"sady"

Topic: 3 
Words: 0.102*"nowy" + 0.067*"warszawa" + 0.067*"park" + 0.044*"bezpieczny" + 0.038*"warsztat" + 0.028*"oraz" + 0.021*"przyjazny" + 0.018*"nie" + 0.017*"dzielnicowy" + 0.016*"miejski"

Topic: 4 
Words: 0.128*"zielony" + 0.037*"chodnik" + 0.025*"ursus" + 0.023*"to" + 0.023*"spotkanie" + 0.023*"zabawić" + 0.021*"remonta" + 0.020*"wolski" + 0.020*"remont" + 0.017*"pies"

Topic: 5 
Words: 0.068*"do" + 0.068*"plac" + 0.061*"

In [127]:
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

In [128]:
lda_model = gensim.models.LdaMulticore(corpus_tfidf, num_topics=5, id2word=vocab, passes=2, workers=10)

In [129]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.036*"pieszy" + 0.022*"nowa" + 0.021*"bezpieczny" + 0.020*"dzielnica" + 0.019*"praga" + 0.018*"bezpłatny" + 0.018*"biblioteka" + 0.016*"warsztaty" + 0.013*"oraz" + 0.012*"numer"
Topic: 1 
Words: 0.223*"na" + 0.062*"dla" + 0.044*"ulica" + 0.038*"park" + 0.025*"plac" + 0.017*"zabawa" + 0.014*"plenerowy" + 0.011*"drzewo" + 0.008*"po" + 0.008*"remont"
Topic: 2 
Words: 0.060*"zielony" + 0.039*"przy" + 0.028*"dorosły" + 0.027*"mini" + 0.023*"rowerowy" + 0.016*"warsztat" + 0.014*"on" + 0.014*"wzdłuż" + 0.012*"edukacyjny" + 0.012*"nr"
Topic: 3 
Words: 0.049*"dziecko" + 0.039*"do" + 0.030*"młodzież" + 0.026*"zieleń" + 0.026*"szkoła" + 0.023*"zajęcie" + 0.019*"stary" + 0.015*"mały" + 0.013*"nowość" + 0.013*"zabawić"
Topic: 4 
Words: 0.044*"nowy" + 0.033*"ul" + 0.030*"warszawa" + 0.020*"wola" + 0.019*"zająć" + 0.015*"chodnik" + 0.014*"podstawowy" + 0.012*"kultura" + 0.011*"zielone" + 0.009*"miejsce"


In [130]:
tags = ['prep', 'conj', 'interj', 'part']
df_waw_clean = df_waw3[~df_waw3['tag'].str.contains('|'.join(tags))]
df_waw_clean.sample(10)

,rok,miasto,nazwa,start_ind,end_ind,form,lemma,tag,qualifiers,_,lemma_clean,tokens
83613,2020,WARSZAWA,Zmiana nawierzchni i instalacja oświetlenia - dotyczy ogólnodostępnego boiska szkolnego do piłki nożnej przy SP 124,8,9,boiska,boisko,subst:pl:nom.acc.voc:n:ncol,['nazwa_pospolita'],[],boisko,[boisko]
82600,2020,WARSZAWA,"W kinie w Aninie - adaptacja sali, bezpłatne seanse i spotkania z twórcami",6,7,sali,sala,subst:sg:dat.loc:f,['nazwa_pospolita'],[],sala,[sala]
51233,2019,WARSZAWA,Punkt porad wychowawczych,0,1,Punkt,punkt,subst:sg:nom.acc:m3,['nazwa_pospolita'],[],punkt,[punkt]
84576,2020,WARSZAWA,Jestem aktywny i mogę zmienić świat – czyli warsztaty dla zdobywców świata (warsztaty rozwojowe dla młodzieży 16-24),8,9,warsztaty,warsztaty,subst:pl:nom.acc.voc:n:pt,['nazwa_pospolita'],[],warsztaty,[warsztaty]
46933,2019,WARSZAWA,Podpórki rowerowe na skrzyżowaniu Żelaznej i Prostej,4,5,Żelaznej,żelazny,adj:sg:loc:f:pos,[],[],żelazny,[żelazny]
35370,2019,WARSZAWA,Modernizacja placu zabaw przy Szkole Podstawowej nr 293 im. J. Kochanowskiego,5,6,Podstawowej,podstawowy,adj:sg:loc:f:pos,[],[],podstawowy,[podstawowy]
7805,2018,WARSZAWA,Budowa brakującego odcinka chodnika przy rondzie Widawska/Księcia Bolesława,0,1,Budowa,Budowo,subst:sg:gen:n:ncol,['nazwa_geograficzna'],[],Budowo,[budowo]
12366,2018,WARSZAWA,Doposażenie siłowni plenerowej w urządzenia do street workoutu na terenie zieleńca zlokalizowanego przy MHŻP POLIN.,2,3,plenerowej,plenerowy,adj:sg:loc:f:pos,[],[],plenerowy,[plenerowy]
38748,2019,WARSZAWA,Miejsce Przyjazne Mieszkańcom Wilanowa - Zielony plac zabaw i miejsce spotkań mieszkańców na planie „Klucza Dóbr Wilanowskich”,7,8,zabaw,zabawa,subst:pl:gen:f,['nazwa_pospolita'],[],zabawa,[zabawa]
36177,2019,WARSZAWA,"Usuwanie barier na rowerowym ""Słonecznym Szlaku"". Zniwelowanie wystającej komory telekomunikacyjnej",1,2,barier,bariera,subst:pl:gen:f,['nazwa_pospolita'],[],bariera,[bariera]


In [131]:
processed_docs = df_waw_clean['tokens']
vocab = gensim.corpora.Dictionary(processed_docs)
bow_corpus = [vocab.doc2bow(doc) for doc in processed_docs]

lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=5, id2word=vocab, passes=2)

for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))
    print("")

Topic: 0 
Words: 0.039*"park" + 0.021*"warsztat" + 0.017*"on" + 0.015*"muzyczny" + 0.013*"sportowy" + 0.012*"spotkanie" + 0.012*"skwer" + 0.012*"zielona" + 0.011*"przyjazny" + 0.011*"wole"

Topic: 1 
Words: 0.071*"zielony" + 0.057*"dziecko" + 0.040*"plac" + 0.027*"zająć" + 0.026*"mini" + 0.025*"nowa" + 0.024*"dzielnica" + 0.021*"plenerowy" + 0.018*"praga" + 0.018*"kultura"

Topic: 2 
Words: 0.076*"wola" + 0.063*"ulica" + 0.041*"pieszy" + 0.020*"bezpłatny" + 0.018*"sport" + 0.013*"dom" + 0.012*"ursus" + 0.012*"lęgowy" + 0.012*"rodzinny" + 0.011*"żoliborski"

Topic: 3 
Words: 0.053*"nowy" + 0.041*"ul" + 0.028*"dorosły" + 0.025*"zajęcie" + 0.022*"biblioteka" + 0.014*"boisko" + 0.012*"edukacyjny" + 0.012*"zabawić" + 0.011*"ursynów" + 0.011*"oświetlić"

Topic: 4 
Words: 0.036*"warszawa" + 0.035*"szkoła" + 0.033*"zieleń" + 0.031*"młodzież" + 0.028*"podstawowy" + 0.022*"bezpieczny" + 0.022*"woli" + 0.020*"warsztaty" + 0.019*"do" + 0.019*"chodnik"



In [132]:
#tfidf
processed_docs = df_waw_clean['tokens']
vocab = gensim.corpora.Dictionary(processed_docs)
corpus = [vocab.doc2bow(doc) for doc in processed_docs]
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

lda_model = gensim.models.LdaMulticore(corpus_tfidf, num_topics=5, id2word=vocab, passes=2, workers=10)

for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.058*"zielony" + 0.037*"plac" + 0.034*"młodzież" + 0.025*"zajęcie" + 0.025*"do" + 0.022*"bezpłatny" + 0.020*"podstawowy" + 0.017*"warsztaty" + 0.016*"drzewo" + 0.013*"sportowy"
Topic: 1 
Words: 0.024*"szkoła" + 0.019*"bezpieczny" + 0.017*"chodnik" + 0.014*"kultura" + 0.012*"on" + 0.012*"zabawić" + 0.012*"wawerski" + 0.010*"zielone" + 0.009*"angielski" + 0.009*"przejść"
Topic: 2 
Words: 0.042*"pieszy" + 0.032*"dorosły" + 0.028*"mini" + 0.022*"biblioteka" + 0.018*"stary" + 0.017*"praga" + 0.012*"dorosła" + 0.012*"miejsce" + 0.012*"spotkanie" + 0.012*"zajęcia"
Topic: 3 
Words: 0.059*"nowy" + 0.036*"ul" + 0.033*"warszawa" + 0.023*"zająć" + 0.022*"nowa" + 0.022*"dzielnica" + 0.018*"warsztat" + 0.013*"dom" + 0.012*"edukacyjny" + 0.011*"sport"
Topic: 4 
Words: 0.057*"ulica" + 0.043*"dziecko" + 0.041*"park" + 0.036*"wola" + 0.027*"zieleń" + 0.023*"rowerowy" + 0.021*"zabawa" + 0.017*"plenerowy" + 0.012*"mały" + 0.012*"nr"


In [133]:
# Wizualizacja tematów
vis_data = vis.prepare(lda_model, corpus_tfidf, vocab)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis_data)

In [134]:
stopwords = ['do', 'wraz', 'oraz', 'ul', 'ulica', 'nr', 'numer', 'to', 'on', 'przy', 'dla', 'na', 'po','jak', 'ten', 'nie', 'od', 'oda', 'czyli']
df_waw_clean4 = df_waw3[~df_waw3['lemma'].str.contains('|'.join(stopwords))]
df_waw_clean4.sample(10)

,rok,miasto,nazwa,start_ind,end_ind,form,lemma,tag,qualifiers,_,lemma_clean,tokens
65791,2020,WARSZAWA,Białołęckie place zabaw przyjazne dla dzieci z niepełnosprawnościami,5,6,dzieci,dziecko,subst:pl:gen:n:col,['nazwa_pospolita'],[],dziecko,[dziecko]
67646,2020,WARSZAWA,Bielany na Orbicie - spotkania z kosmosem dla młodzieży i dorosłych,0,1,Bielany,Bielany,subst:pl:nom.acc.voc:n:pt,['nazwa_geograficzna'],[],Bielany,[bielany]
83983,2020,WARSZAWA,Wawerski Klub Ojców – Integracja środowiskowa ojców i dzieci,1,2,Klub,klub,subst:sg:nom.acc:m3,['nazwa_pospolita'],[],klub,[klub]
11003,2018,WARSZAWA,Dąb za dąb - sadzimy duży dąb przy Domu Kultury Świt,6,7,dąb,dąb,subst:sg:nom.acc:m3,['nazwa_pospolita'],[],dąb,[dąb]
76565,2020,WARSZAWA,Potańcówki z muzyką swingową na żywo,3,4,swingową,swingowy,adj:sg:inst:f:pos,[],[],swingowy,[swingowy]
21545,2018,WARSZAWA,Dzielnicowe Potańcówki Pod Chmurką,0,1,Dzielnicowe,dzielnicowy:A,adj:sg:nom.voc:n:pos,[],[],dzielnicowy,[dzielnicowy]
6465,2018,WARSZAWA,Dziecko z natury - warsztaty świadomego rodzicielstwa,4,5,warsztaty,warsztaty,subst:pl:nom.acc.voc:n:pt,['nazwa_pospolita'],[],warsztaty,[warsztaty]
38908,2019,WARSZAWA,"Przedstawienia teatralne dla dzieci w Bibliotece ""Odolanka""",0,1,Przedstawienia,przedstawić,ger:pl:nom.acc:n:perf:aff,[],[],przedstawić,[przedstawić]
52091,2019,WARSZAWA,Wszyscy Twórcy dobrze się bawią - warsztaty dla dzieci z projektowania w przestrzeni.,12,13,przestrzeni,przestrzeń,subst:sg:voc:f,['nazwa_pospolita'],[],przestrzeń,[przestrzeń]
83130,2020,WARSZAWA,Klonikowa akademia – Zajęcia rozwijające Kreatywność Uczniów w SP 216,3,4,Zajęcia,zajęcia,subst:pl:nom.acc.voc:n:pt,['nazwa_pospolita'],[],zajęcia,[zajęcia]


In [135]:
#tfidf
processed_docs = df_waw_clean4['tokens']
vocab = gensim.corpora.Dictionary(processed_docs)
corpus = [vocab.doc2bow(doc) for doc in processed_docs]
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

lda_model = gensim.models.LdaMulticore(corpus_tfidf, num_topics=6, id2word=vocab, passes=2, workers=10)

for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.027*"plenerowy" + 0.022*"zajęcia" + 0.018*"pies" + 0.012*"pieszy" + 0.011*"szkolny" + 0.011*"każdy" + 0.010*"letni" + 0.010*"oświetlić" + 0.010*"bezpieczeństwo" + 0.010*"ścieżka"
Topic: 1 
Words: 0.044*"plac" + 0.040*"zająć" + 0.039*"zieleń" + 0.033*"bezpłatny" + 0.032*"biblioteka" + 0.020*"wzdłuż" + 0.018*"angielski" + 0.018*"centrum" + 0.017*"ursynów" + 0.017*"publiczny"
Topic: 2 
Words: 0.083*"nowy" + 0.057*"park" + 0.032*"dzielnica" + 0.028*"stary" + 0.027*"zabawa" + 0.019*"mały" + 0.019*"edukacyjny" + 0.018*"wawerski" + 0.017*"przejść" + 0.017*"boisko"
Topic: 3 
Words: 0.051*"warszawa" + 0.048*"szkoła" + 0.044*"rowerowy" + 0.038*"zajęcie" + 0.034*"bezpieczny" + 0.031*"praga" + 0.023*"zabawić" + 0.015*"książka" + 0.014*"stara" + 0.013*"miejski"
Topic: 4 
Words: 0.026*"warsztat" + 0.019*"ursus" + 0.015*"muzyczny" + 0.015*"senior" + 0.015*"lęgowy" + 0.014*"sąsiedzki" + 0.014*"sp" + 0.013*"zielenić" + 0.012*"ruch" + 0.012*"nowe"
Topic: 5 
Words: 0.071*"dziecko" + 0.

## Reszta miast

In [136]:
df_roc = pd.read_csv('/content/drive/My Drive/Konkursy Kaggle/Budżet obywatelski/Morfeusz_ROC.csv')
df_roc.head()

,rok,miasto,nazwa,start_ind,end_ind,form,lemma,tag,qualifiers,_
0,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",0,1,Doposażenie,doposażyć,ger:sg:nom.acc:n:perf:aff,[],[]
1,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",1,2,parku,park,subst:sg:gen:m3,['nazwa_pospolita'],[]
2,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",1,2,parku,park,subst:sg:loc:m3,['nazwa_pospolita'],[]
3,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",1,2,parku,park,subst:sg:voc:m3,['nazwa_pospolita'],[]
4,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",2,3,pomiędzy,pomiędzy,prep:acc,[],[]


In [139]:
df_roc['lemma_clean'] = df_roc['lemma'].str.split(':', expand=True)[0]
df_roc.sample(10)

,rok,miasto,nazwa,start_ind,end_ind,form,lemma,tag,qualifiers,_,lemma_clean,tokens
288145,2018/19,ŁÓDŹ,Rewitalizacja skwerku wzdłuż ul. Gojawiczyńskiej (od ul. Felińskiego do ul. Śląskiej).,11,12,do,do:S,subst:sg.pl:nom.gen.dat.acc.inst.loc.voc:n:ncol,['nazwa_pospolita'],['muz.'],do,[do]
309214,2019,ŁÓDŹ,Tężnia solankowa w parku pomiędzy ul. Lutomierską a ul. Drewnowską.,7,8,Lutomierską,lutomierski,adj:sg:inst:f:pos,[],[],lutomierski,[lutomierski]
176872,2019,KALISZ,Zakup sprzętu sportowego dla zespołów młodzieżowych MKS Kalisz.,8,9,.,.,interp,[],[],.,[]
248874,2019,GRUDZIĄDZ,Budowa nowoczesnego aktywnego przejścia dla pieszych na ul. Konstytucji 3 Maja - przy parafii pw. Świętego Stanisława Biskupa i Męczennika w Grudziądzu,0,1,Budowa,Budowo,subst:pl:nom.acc.voc:n:ncol,['nazwa_geograficzna'],[],Budowo,[budowo]
256161,2019,TORUŃ,Inwentaryzacja i waloryzacja cennych drzew na terenie Rudaka.,5,6,na,na:P,prep:acc,[],[],na,[na]
171179,2018,OLSZTYN,114. Budowa schodów przy kapliczce z ulicy starej Bałtyckiej (dół) do ulicy Bałtyckiej (góra) przystanek autobusowy ul. Letniskowa,19,20,przystanek,przystanek,subst:sg:nom.acc:m3,['nazwa_pospolita'],[],przystanek,[przystanek]
250886,2019,GRUDZIĄDZ,Wymiana chodnika przed blokiem Kopernika 2,4,5,Kopernika,Kopernik:Sm1,subst:sg:gen.acc:m1,['nazwisko'],[],Kopernik,[kopernik]
318805,2019,ŁÓDŹ,RADOSNY PLAC ZABAW- remont i doposażenie placu zabaw przy SP 109 im. Ludwiki Wawrzyńskiej.,10,11,SP,SP,subst:sg.pl:nom.gen.dat.acc.inst.loc.voc:f,['nazwa_organizacji'],[],SP,[sp]
54441,2018,BIELSKO-BIAŁA,Bezpieczeństwo przy kościele św. Barbary w Mikuszowicach Krakowskich ul. Cyprysowa,5,6,Barbary,Barbara:Sm1,depr:pl:nom.acc.voc:m2,['nazwisko'],[],Barbara,[barbara]
113545,2018,KATOWICE,"M/2/V - Dni Seniora (na) Fest W Bogucicach i Zawodziu - warsztaty artystyczne, spotkania integracyjne wraz Z koncertami dla Seniorów Z Dzielnic Bogucice i Zawodzie",30,31,i,i:C,conj,[],[],i,[]


In [140]:
df_roc['tokens'] = df_roc['lemma_clean'].apply(simple_preprocess)
df_roc.sample(10)

,rok,miasto,nazwa,start_ind,end_ind,form,lemma,tag,qualifiers,_,lemma_clean,tokens
211355,2018,JELENIA GÓRA,Wielofunkcyjne boisko do sportów zręcznościowych i precyzyjnych,0,1,Wielofunkcyjne,wielofunkcyjny,adj:sg:nom.voc:n:pos,[],[],wielofunkcyjny,[wielofunkcyjny]
159493,2020,KIELCE,"""Port Wodny Kielce"" - koncerty, konkursy, rejsy. Dwa wakacyjne festyny rodzinne nad Zalewem.",13,14,wakacyjne,wakacyjny,adj:sg:acc:n:pos,[],[],wakacyjny,[wakacyjny]
88081,2020,KATOWICE,Ogród Śląskich Noblistów,1,2,Śląskich,Śląski,subst:pl:gen.acc:m1,['nazwisko'],[],Śląski,[śląski]
36906,2018,GDAŃSK,Uporządkowanie i utwardzenie terenu pomiędzy Szkołą Podstawową nr 43 (dawniej Gimnazjum nr 29) a posesją przy ul. Beethovena 22 w Gdańsku wraz z utworzeniem miejsca pod mural lub graffiti,30,31,lub,lub,conj,[],[],lub,[lub]
278326,2020,GORZÓW WLKP.,Remont schodów między blokami przy ul. Gwiaździstej 18-20,1,2,schodów,schód,subst:pl:gen:m3,['nazwa_pospolita'],[],schód,[schód]
138536,2019,SOSNOWIEC,Zakup nowosci wydawniczych dla filii nr 14 Zagórze,7,8,Zagórze,zagórze,subst:sg:nom.acc.voc:n:ncol,['nazwa_pospolita'],[],zagórze,[zagórze]
364018,2020,PIOTRKÓW TRYBUNALSKI,Bieżnia 4-torowa na boisku szkolnym w SP nr 12 im. K. Makuszyńskiego.,11,12,K,kompania,brev:npun,[],['wojsk.'],kompania,[kompania]
361339,2020,ŁÓDŹ,GABINETY PRZYJAZNE PACJENTOM - REMONT I WYPOSAŻENIE GABINETÓW W PORADNI POZ PRZYCHODNI ŚRÓDMIEŚCIE.,3,4,-,-,interp,[],[],-,[]
394848,2018,RZESZÓW,"Wykonanie miejsc postojowych wzdłuż ul. Krzywoustego od strony budynków przy ul. Krzywoustego 1, 3, 5, 7.",12,13,.,.,interp,[],[],.,[]
296147,2018/19,ŁÓDŹ,Czyste chodniki – kosze na psie odchody na Starym Polesiu.,8,9,Starym,Stary:Sm1,subst:sg:loc:m1,['nazwisko'],[],Stary,[stary]


In [141]:
df_roc3 = df_roc[df_roc["tokens"].str.len() != 0]
df_roc3.sample(10)

,rok,miasto,nazwa,start_ind,end_ind,form,lemma,tag,qualifiers,_,lemma_clean,tokens
167932,2019,OLSZTYN,110. STWORZENIE STANOWISK KOMPUTEROWYCH W FILII NR 5 BIBLIOTEKI MIEJSKIEJ W OLSZTYNIE (ORŁOWICZA 27),12,13,OLSZTYNIE,Olsztyn:Sm3,subst:sg:loc:m3,['nazwa_geograficzna'],[],Olsztyn,[olsztyn]
193160,2019,POZNAŃ,Rekreacja na Strzeszynie - osiedlowy pawilon,1,2,na,na:I,interj,[],[],na,[na]
83805,2020,GLIWICE,Wynajem sali przy Placu Jasminu 2 dla potrzeb organizacji zajeó sportowo-rekreacyjnych.,12,13,rekreacyjnych,rekreacyjny,adj:pl:gen:m1.m2.m3.f.n:pos,[],['książk.'],rekreacyjny,[rekreacyjny]
101781,2020,KATOWICE,"W ramach zadania nalezy dokonac zakupu 1 okazu magnolii. Proponuje siq magnoliq Loebnera typu Leonard Messel. Jezeli W ocenie Zespotu Opiniujacego ten gatunek okaze sie niezasadny lub niemozliwy do posadzenia i utrzymania, proponuje siq inny okaz Z rodziny magnoliowatych. Przy zakupie nalezy wywazyé fakt trudnego znoszenia przesadzania przez magnolie (czyli okaz nie moze byó ""za stary""), Z pró...",105,106,co,co:M,comp,[],[],co,[co]
26585,2020,GDAŃSK,FIT PARK - ŻABIANKA ZAWSZE W DOBREJ FORMIE,0,1,FIT,Fit:Sm1,subst:sg:nom:m1,['nazwisko'],[],Fit,[fit]
316175,2019,ŁÓDŹ,Bezpieczna droga do SP 206.,1,2,droga,drogi:A,adj:sg:nom.voc:f:pos,[],[],drogi,[drogi]
197981,2019,KOSZALIN,2. Fontanna na Skwerze PCK w 100-lecie istnienia Polskiego Czerwonego Krzyża,10,11,Czerwonego,czerwone,subst:sg:gen:n:ncol,['nazwa_pospolita'],[],czerwone,[czerwone]
209541,2019,ŚWINOUJŚCIE,Karsibór. System do monitorowania wybranych obszarów prawobrzeza Z jednostka rejestrujaca obrazy ze wszystkich kamer. Poprawa bezpieczenstwa na obszarach monitorowanych. Zauwazalny monitoring efektywnie odstrasza przed rozbojami.,18,19,na,na:P,prep:loc,[],[],na,[na]
67272,2019,BYTOM,"Bytom miastem karate - otwarte treningi (warsztaty), konkurs Z nagrodami dla osób które je ukoncza, gry i zabawy W trakcie trwania wydarzenia, Mistrzostwa Bytomia W Karate Tradycyjnym - organizacja, seminarium Z najwybitniejszym polskim karetka, W hali na Skarpie",38,39,polskim,polskie,subst:sg:loc:n:ncol,['nazwa_pospolita'],['przest.'],polskie,[polskie]
368016,2018,KRAKÓW,Nie daj się zalać!!!,2,3,się,się,part,[],[],się,[się]


In [142]:
#cbow
processed_docs = df_roc3['tokens']
vocab = gensim.corpora.Dictionary(processed_docs)
bow_corpus = [vocab.doc2bow(doc) for doc in processed_docs]

lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=5, id2word=vocab, passes=2)

for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))
    print("")

Topic: 0 
Words: 0.220*"na" + 0.056*"park" + 0.047*"do" + 0.030*"nowa" + 0.027*"łódź" + 0.024*"dziecko" + 0.020*"wolny" + 0.015*"huta" + 0.013*"parking" + 0.010*"modernizacja"

Topic: 1 
Words: 0.053*"ul" + 0.052*"dla" + 0.048*"plac" + 0.024*"chodnik" + 0.021*"zielone" + 0.020*"podstawowy" + 0.020*"budowa" + 0.016*"oraz" + 0.013*"boisko" + 0.012*"to"

Topic: 2 
Words: 0.036*"szkoła" + 0.036*"miejski" + 0.030*"budowo" + 0.028*"centrum" + 0.027*"zabawa" + 0.024*"dzielnica" + 0.017*"miasto" + 0.017*"sp" + 0.016*"remont" + 0.016*"remonta"

Topic: 3 
Words: 0.091*"zielony" + 0.074*"ulica" + 0.035*"mini" + 0.024*"pieszy" + 0.022*"numer" + 0.022*"nr" + 0.022*"sportowy" + 0.021*"zabawić" + 0.018*"teren" + 0.016*"on"

Topic: 4 
Words: 0.066*"nowy" + 0.043*"przy" + 0.041*"osiedle" + 0.026*"bezpieczny" + 0.020*"nawierzchnia" + 0.017*"stary" + 0.013*"zieleń" + 0.012*"rowerowy" + 0.012*"mały" + 0.010*"zielona"



In [143]:
#tfidf
processed_docs = df_roc3['tokens']
vocab = gensim.corpora.Dictionary(processed_docs)
corpus = [vocab.doc2bow(doc) for doc in processed_docs]
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

lda_model = gensim.models.LdaMulticore(corpus_tfidf, num_topics=5, id2word=vocab, passes=2, workers=10)

for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.052*"nowy" + 0.048*"dla" + 0.048*"przy" + 0.048*"park" + 0.029*"pieszy" + 0.023*"bezpieczny" + 0.022*"numer" + 0.020*"nawierzchnia" + 0.020*"centrum" + 0.019*"teren"
Topic: 1 
Words: 0.047*"plac" + 0.023*"dziecko" + 0.023*"budowa" + 0.020*"zabawić" + 0.018*"podstawowy" + 0.017*"remont" + 0.016*"parkingowy" + 0.015*"sp" + 0.013*"parking" + 0.012*"mały"
Topic: 2 
Words: 0.210*"na" + 0.068*"zielony" + 0.034*"budowo" + 0.027*"zabawa" + 0.021*"nr" + 0.020*"sportowy" + 0.015*"zielone" + 0.014*"drogi" + 0.014*"miejsce" + 0.014*"zieleń"
Topic: 3 
Words: 0.087*"ulica" + 0.053*"do" + 0.023*"chodnik" + 0.007*"oświetlenie" + 0.007*"postojowy" + 0.006*"ten" + 0.006*"przyjazny" + 0.006*"aktywność" + 0.006*"publiczny" + 0.006*"edukacyjny"
Topic: 4 
Words: 0.067*"ul" + 0.034*"miejski" + 0.033*"mini" + 0.032*"łódź" + 0.030*"szkoła" + 0.028*"osiedle" + 0.023*"oraz" + 0.020*"nowa" + 0.020*"rowerowy" + 0.016*"dzielnica"


In [145]:
tags = ['prep', 'conj', 'interj', 'part', 'adv']
df_roc_clean = df_roc3[~df_roc3['tag'].str.contains('|'.join(tags))]
df_roc_clean.sample(10)

,rok,miasto,nazwa,start_ind,end_ind,form,lemma,tag,qualifiers,_,lemma_clean,tokens
121524,2018,KATOWICE,L22/4/V - Podlesie jak malowane. Warsztaty malarsko - rekodzielnicze,8,9,malowane,malować,ppas:sg:nom.acc.voc:n:imperf:aff,[],[],malować,[malować]
338025,2019,ŁÓDŹ,Chodniki na Ormiańskiej i Johna Wayne'a.,0,1,Chodniki,Chodnik:Sm1,depr:pl:nom.acc.voc:m2,['nazwisko'],[],Chodnik,[chodnik]
353048,2020,ŁÓDŹ,"Wymiana nawierzchni na placu szkolnym w SP4 w Łodzi, ul. Milionowa 6",8,9,Łodzi,Łodzia,subst:sg:dat.loc:f,['nazwa_geograficzna'],[],Łodzia,[łodzia]
411919,2020,SUWAŁKI,Remont ul. Sianożęć na wysokości ROD „Malwa”,3,4,Sianożęć,sianożęć,subst:sg:acc:f,['nazwa_pospolita'],['przest.'],sianożęć,[sianożęć]
95525,2020,KATOWICE,Zakup nowości wydawniczych dla Filii 33 Miejskiej Biblioteki Publicznej w Katowicach-Bogucicach,1,2,nowości,nowość,subst:pl:gen:f,['nazwa_pospolita'],[],nowość,[nowość]
65717,2019,BYTOM,"Ogólnodostepne boisko wielofunkcyjne przy ulicy Orlat Lwowskich - przy SP nr 40 - nowoczesne wielofunkcyjne boisko do pilki recznej, siatkowej, noznej i koszykowej Z wykorzystaniem do lekkiej atletyki i gier i zabaw sportowych",6,7,Lwowskich,Lwowska,subst:pl:gen:f,['nazwisko'],[],Lwowska,[lwowska]
252595,2018,TORUŃ,Nowy chodnik i parking,1,2,chodnik,chodnik,subst:sg:nom.acc:m3,['nazwa_pospolita'],[],chodnik,[chodnik]
395254,2018,RZESZÓW,Remont i modernizacja placu zabaw przy budynku Przedszkola Publicznego nr 19 w Rzeszowie przy al. prof. A. Krzyżanowskiego 20.,18,19,A,accusativus,brev:pun,[],['jęz.'],accusativus,[accusativus]
407371,2020,BIAŁYSTOK,SP 34 i PS 49 wolne od bakterii i wirusów,9,10,wirusów,wirus,subst:pl:gen:m2,['nazwa_pospolita'],[],wirus,[wirus]
126095,2020,MYSŁOWICE,"Wymiana lub remont zniszczonego chodnika , nierównego, ze starymi płytami chodnikowymi i wybrukowanie miejsc na postój samochodów",3,4,zniszczonego,zniszczony,adj:sg:acc:m1.m2:pos,[],[],zniszczony,[zniszczony]


In [146]:
#cbow
processed_docs = df_roc_clean['tokens']
vocab = gensim.corpora.Dictionary(processed_docs)
bow_corpus = [vocab.doc2bow(doc) for doc in processed_docs]


lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=5, id2word=vocab, passes=2)

for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))
    print("")

Topic: 0 
Words: 0.073*"nowy" + 0.035*"zielone" + 0.034*"nowa" + 0.031*"budowo" + 0.028*"bezpieczny" + 0.026*"do" + 0.025*"pieszy" + 0.023*"nr" + 0.020*"zielona" + 0.019*"sp"

Topic: 1 
Words: 0.047*"osiedle" + 0.040*"miejski" + 0.040*"szkoła" + 0.038*"mini" + 0.022*"budowa" + 0.022*"podstawowy" + 0.012*"modernizacja" + 0.011*"aktywność" + 0.011*"biblioteka" + 0.010*"orlik"

Topic: 2 
Words: 0.030*"zabawa" + 0.024*"numer" + 0.018*"stary" + 0.015*"rowerowy" + 0.013*"mały" + 0.012*"miejsce" + 0.011*"aleja" + 0.010*"stare" + 0.010*"plenerowy" + 0.009*"przedszkole"

Topic: 3 
Words: 0.127*"zielony" + 0.074*"ulica" + 0.030*"dzielnica" + 0.025*"dziecko" + 0.022*"zabawić" + 0.016*"dobry" + 0.016*"remont" + 0.013*"miasto" + 0.009*"zdrowy" + 0.008*"nowe"

Topic: 4 
Words: 0.063*"park" + 0.057*"ul" + 0.052*"plac" + 0.032*"łódź" + 0.029*"centrum" + 0.026*"chodnik" + 0.023*"nawierzchnia" + 0.023*"sportowy" + 0.021*"wolny" + 0.018*"zieleń"



In [147]:
#tfidf
processed_docs = df_roc_clean['tokens']
vocab = gensim.corpora.Dictionary(processed_docs)
corpus = [vocab.doc2bow(doc) for doc in processed_docs]
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

lda_model = gensim.models.LdaMulticore(corpus_tfidf, num_topics=5, id2word=vocab, passes=2, workers=10)

for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.073*"ul" + 0.038*"mini" + 0.034*"szkoła" + 0.030*"do" + 0.025*"chodnik" + 0.021*"nowa" + 0.019*"remont" + 0.018*"parkingowy" + 0.017*"dzielnica" + 0.016*"drogi"
Topic: 1 
Words: 0.053*"plac" + 0.025*"dziecko" + 0.025*"budowa" + 0.022*"centrum" + 0.015*"miejsce" + 0.012*"młodzież" + 0.011*"sala" + 0.011*"wolny" + 0.010*"przedszkole" + 0.009*"stare"
Topic: 2 
Words: 0.056*"nowy" + 0.037*"łódź" + 0.030*"zabawa" + 0.022*"zabawić" + 0.020*"teren" + 0.020*"podstawowy" + 0.019*"stary" + 0.017*"sp" + 0.016*"zielone" + 0.015*"zieleń"
Topic: 3 
Words: 0.099*"ulica" + 0.052*"park" + 0.039*"miejski" + 0.033*"pieszy" + 0.031*"osiedle" + 0.026*"bezpieczny" + 0.024*"nr" + 0.023*"nawierzchnia" + 0.019*"remonta" + 0.012*"budynek"
Topic: 4 
Words: 0.072*"zielony" + 0.038*"budowo" + 0.024*"numer" + 0.023*"sportowy" + 0.021*"rowerowy" + 0.016*"boisko" + 0.014*"parking" + 0.012*"biblioteka" + 0.011*"zakup" + 0.011*"zakupić"


In [148]:
# Wizualizacja tematów
vis_data = vis.prepare(lda_model, corpus_tfidf, vocab)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis_data)

In [150]:
stopwords = ['do', 'wraz', 'oraz', 'ul', 'ulica', 'nr', 'numer', 'to', 'on', 'przy', 'dla', 'na', 'po','jak', 'ten', 'nie', 'od', 'oda', 'czyli', 'ii']
df_roc_clean4 = df_roc3[~df_roc3['lemma'].str.contains('|'.join(stopwords))]
df_roc_clean4.sample(10)

,rok,miasto,nazwa,start_ind,end_ind,form,lemma,tag,qualifiers,_,lemma_clean,tokens
14097,2019,GDYNIA,"Rewitalizacja placu zabaw oraz budowa sitowni miejskiej dla dorostych pomiedzy ulicami Eugeniusza Kwiatkowskiego 100, 102 i Ptk. Stanistawa Dabka 251, 257",6,7,miejskiej,miejski,adj:sg:dat:f:pos,[],[],miejski,[miejski]
224506,2019,WROCŁAW,Porządkowanie parkowania na ul. Pocztowej,1,2,parkowania,parkować,ger:sg:gen:n:imperf:aff,[],[],parkować,[parkować]
277954,2020,GORZÓW WLKP.,Budowa drogi na istniejącej podbudowie przy ul. Brukselskiej,1,2,drogi,drogi:A,adj:sg:acc:m3:pos,[],[],drogi,[drogi]
259238,2019,WŁOCŁAWEK,"SPORT I REKREACJA - PROJEKT ""ZDROWE ZAZAMCZE""",6,7,ZDROWE,Zdrowa,subst:pl:nom.acc.voc:f,['nazwa_geograficzna'],[],Zdrowa,[zdrowa]
12422,2019,GDYNIA,Gry i zabawy ruchowe dla najmfodszych Z elementami nauki jezyków obcych W dzielnicy Dabrowa,12,13,dzielnicy,dzielnica,subst:sg:gen:f,['nazwa_pospolita'],[],dzielnica,[dzielnica]
130075,2018,RYBNIK,"Renowacja bocznego boiska KS Rymer wraz Z jego otoczeniem. Pielegnacja i renowacja murawy, montaz instalacji oswietlenia, zaprojektowanie i montaz matej architektury (lawki, fawy, kosze, parkingi rowerowe, stupki, donice, pergole, trejaze, kominek grillowy, zamykana piaskownica). Montaz sifowni (6 urzadzen) na wolnym powietrzu. Organizacja kina ""pod chmurka"".",22,23,matej,Mateja:Sf,subst:pl:gen:f,['nazwisko'],['char.'],Mateja,[mateja]
184122,2019,KONIN,BEZPIECZNE PRZEJSCIA DLA PIESZYCH W REGJONIE SZKOLY PODSTAWOWEJ NR 15 W KONINIE,3,4,PIESZYCH,pieszy:A,adj:pl:gen:m1.m2.m3.f.n:pos,[],[],pieszy,[pieszy]
322172,2019,ŁÓDŹ,Biegacze kontra SMOG na Rudzie.,4,5,Rudzie,ruda:Sf~y,subst:sg:dat.loc:f,['nazwa_pospolita'],['geol.'],ruda,[ruda]
96032,2020,KATOWICE,Pasy rowerowe na ul. Sokolskiej od ul. Misjonarzy Oblatów do ul. Chorzowskiej,14,15,Chorzowskiej,chorzowski,adj:sg:dat:f:pos,[],[],chorzowski,[chorzowski]
237270,2020,BYDGOSZCZ,Budowa chodnika 45m obok Jaru Czynu Społecznego i rewitalizacja trasy pieszo-rowerowej na Jarze,15,16,Jarze,jar,subst:sg:loc:m3,['nazwa_pospolita'],[],jar,[jar]


In [151]:
#tfidf
processed_docs = df_roc_clean4['tokens']
vocab = gensim.corpora.Dictionary(processed_docs)
corpus = [vocab.doc2bow(doc) for doc in processed_docs]
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

lda_model = gensim.models.LdaMulticore(corpus_tfidf, num_topics=6, id2word=vocab, passes=2, workers=10)

for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.059*"łódź" + 0.053*"zabawa" + 0.049*"osiedle" + 0.027*"zieleń" + 0.018*"zakupić" + 0.016*"stare" + 0.013*"pies" + 0.012*"publiczny" + 0.012*"edukacyjny" + 0.011*"nowoczesny"
Topic: 1 
Words: 0.095*"nowy" + 0.064*"miejski" + 0.056*"szkoła" + 0.038*"zabawić" + 0.029*"boisko" + 0.026*"miejsce" + 0.019*"sala" + 0.015*"oświetlić" + 0.012*"wielofunkcyjny" + 0.012*"aktywność"
Topic: 2 
Words: 0.043*"dziecko" + 0.029*"parkingowy" + 0.028*"sp" + 0.026*"dzielnica" + 0.023*"mały" + 0.019*"rekreacyjny" + 0.017*"filia" + 0.016*"przedszkole" + 0.013*"rekreacja" + 0.012*"aleja"
Topic: 3 
Words: 0.088*"plac" + 0.062*"mini" + 0.053*"pieszy" + 0.042*"bezpieczny" + 0.038*"centrum" + 0.034*"teren" + 0.032*"stary" + 0.019*"budynek" + 0.019*"zakup" + 0.018*"wolny"
Topic: 4 
Words: 0.091*"park" + 0.027*"drogi" + 0.025*"parking" + 0.020*"biblioteka" + 0.015*"osiedlowy" + 0.012*"zająć" + 0.012*"szkolny" + 0.011*"za" + 0.009*"wielki" + 0.009*"przejście"
Topic: 5 
Words: 0.036*"rowerowy" + 0.0

## 2018

In [152]:
df_2018 = pd.read_csv('/content/drive/My Drive/Konkursy Kaggle/Budżet obywatelski/Morfeusz_2018.csv')
df_2018.head()

,rok,miasto,nazwa,start_ind,end_ind,form,lemma,tag,qualifiers,_
0,2018,GDYNIA,"Zagospodarowanie terenu po bytym baraku przy ulicy Dedala - stworzenie sciezek, ustawienie fawek, nasadzenia zieleni i aranzacja matej architektury",0,1,Zagospodarowanie,zagospodarować,ger:sg:nom.acc:n:perf:aff,[],[]
1,2018,GDYNIA,"Zagospodarowanie terenu po bytym baraku przy ulicy Dedala - stworzenie sciezek, ustawienie fawek, nasadzenia zieleni i aranzacja matej architektury",1,2,terenu,teren,subst:sg:gen:m3,['nazwa_pospolita'],[]
2,2018,GDYNIA,"Zagospodarowanie terenu po bytym baraku przy ulicy Dedala - stworzenie sciezek, ustawienie fawek, nasadzenia zieleni i aranzacja matej architektury",2,3,po,po,prep:acc,[],[]
3,2018,GDYNIA,"Zagospodarowanie terenu po bytym baraku przy ulicy Dedala - stworzenie sciezek, ustawienie fawek, nasadzenia zieleni i aranzacja matej architektury",2,3,po,po,prep:dat,[],[]
4,2018,GDYNIA,"Zagospodarowanie terenu po bytym baraku przy ulicy Dedala - stworzenie sciezek, ustawienie fawek, nasadzenia zieleni i aranzacja matej architektury",2,3,po,po,prep:loc,[],[]


In [153]:
df_2018['lemma_clean'] = df_2018['lemma'].str.split(':', expand=True)[0]
df_2018['tokens'] = df_2018['lemma_clean'].apply(simple_preprocess)
df_2018_ = df_2018[df_2018["tokens"].str.len() != 0]

In [154]:
#tfidf
processed_docs = df_2018_['tokens']
vocab = gensim.corpora.Dictionary(processed_docs)
corpus = [vocab.doc2bow(doc) for doc in processed_docs]
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

lda_model = gensim.models.LdaMulticore(corpus_tfidf, num_topics=5, id2word=vocab, passes=2, workers=10)

for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.084*"ulica" + 0.046*"nowy" + 0.026*"szkoła" + 0.021*"oraz" + 0.020*"nawierzchnia" + 0.017*"miejsce" + 0.013*"młodzież" + 0.010*"warszawa" + 0.010*"modernizacja" + 0.010*"publiczny"
Topic: 1 
Words: 0.036*"mini" + 0.019*"chodnik" + 0.018*"numer" + 0.016*"parkingowy" + 0.013*"drogi" + 0.011*"dorosły" + 0.009*"biblioteka" + 0.008*"od" + 0.008*"rekreacyjny" + 0.007*"rekreacja"
Topic: 2 
Words: 0.057*"przy" + 0.055*"dla" + 0.051*"plac" + 0.028*"zabawa" + 0.020*"centrum" + 0.016*"nowa" + 0.014*"droga" + 0.013*"dzielnica" + 0.011*"parking" + 0.011*"zakup"
Topic: 3 
Words: 0.059*"ul" + 0.059*"do" + 0.049*"zielony" + 0.039*"park" + 0.032*"pieszy" + 0.026*"dziecko" + 0.022*"nr" + 0.019*"osiedle" + 0.019*"rowerowy" + 0.016*"teren"
Topic: 4 
Words: 0.194*"na" + 0.032*"budowo" + 0.029*"miejski" + 0.021*"budowa" + 0.019*"sportowy" + 0.018*"zabawić" + 0.017*"podstawowy" + 0.016*"bezpieczny" + 0.015*"boisko" + 0.014*"on"


In [155]:
tags = ['prep', 'conj', 'interj', 'part']
df_2018_clean = df_2018_[~df_2018_['tag'].str.contains('|'.join(tags))]
df_2018_clean.sample(10)

,rok,miasto,nazwa,start_ind,end_ind,form,lemma,tag,qualifiers,_,lemma_clean,tokens
64444,2018,TORUŃ,"Budowa ścieżki pieszo-rowerowej, długość ścieżki 200 m, szerokość 2 m od ul. Niesiołowskiego nr 8 pomiędzy garażami w kierunku północnym do mostku na strudze Toruńskiej",21,22,garażami,garaż,subst:pl:inst:m3,['nazwa_pospolita'],[],garaż,[garaż]
112298,2018,WARSZAWA,1000 nowych książek do wypożyczalni nr 95 dla dorosłych i młodziezy,1,2,nowych,nowy:A,adj:pl:loc:m1.m2.m3.f.n:pos,[],[],nowy,[nowy]
36534,2018,ELBLĄG,6 Wykonanie utwardzenia terenu wraz Z wyznaczeniem miejsc do parkowania pojazdów. ul. Wiejska,4,5,wraz,wraz:S,subst:sg:nom.acc:m3,['nazwa_pospolita'],['daw.'],wraz,[wraz]
43989,2018,LESZNO,"Budowa biezni, boiska do koszykówki oraz placu do gier i zabaw ruchowych wraz Z zagospodarowaniem terenu.",12,13,ruchowych,ruchowy,adj:pl:loc:m1.m2.m3.f.n:pos,[],[],ruchowy,[ruchowy]
56572,2018,WROCŁAW,Zielona wstęga Krynicka - zieleń przyuliczna wzdłuż Krynickiej 54-92,2,3,Krynicka,Krynicka,subst:sg:nom:f,['nazwisko'],[],Krynicka,[krynicka]
48893,2018,SZCZECIN,Modernizacja boiska sportowego przy SP nr 5,1,2,boiska,boisko,subst:pl:nom.acc.voc:n:ncol,['nazwa_pospolita'],[],boisko,[boisko]
2013,2018,GDYNIA,"Park nad rzeka Kacza W Ortowie - W okolicy ulic Sieradzkiej, Wroclawskiej i Kurpiowskiej",10,11,Sieradzkiej,sieradzki,adj:sg:gen:f:pos,[],[],sieradzki,[sieradzki]
47801,2018,KOSZALIN,11. Multimedialne informatory miejskie z zegarem i ławką,2,3,Multimedialne,multimedialny,adj:pl:nom.voc:m2.m3.f.n:pos,[],[],multimedialny,[multimedialny]
24245,2018,KATOWICE,L21/5/V - Bezpiecznie W mojej dzielnicy - zajecia Z samoobrony dla kazdego W MDK Potudnie na Kostuchnie,17,18,MDK,MDK,subst:sg.pl:nom.gen.dat.acc.inst.loc.voc:m3,['nazwa_instytucji'],[],MDK,[mdk]
68737,2018,CHEŁM,Budowa chodnika przy ul. Przemysłowej na odcinku od ul. Rolniczej do ul. Wołyńskiej (po stronie nieparzystej ulicy),0,1,Budowa,Budowo,subst:sg:gen:n:ncol,['nazwa_geograficzna'],[],Budowo,[budowo]


In [156]:
#tfidf
processed_docs = df_2018_clean['tokens']
vocab = gensim.corpora.Dictionary(processed_docs)
corpus = [vocab.doc2bow(doc) for doc in processed_docs]
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

lda_model = gensim.models.LdaMulticore(corpus_tfidf, num_topics=5, id2word=vocab, passes=2, workers=10)

for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.057*"plac" + 0.048*"mini" + 0.023*"nawierzchnia" + 0.023*"podstawowy" + 0.021*"stary" + 0.018*"zieleń" + 0.017*"on" + 0.017*"młodzież" + 0.017*"budowa" + 0.013*"drogi"
Topic: 1 
Words: 0.043*"nowy" + 0.025*"szkoła" + 0.025*"zabawa" + 0.023*"nr" + 0.017*"miejsce" + 0.016*"teren" + 0.013*"mały" + 0.010*"siłownia" + 0.009*"zakupić" + 0.008*"przedszkole"
Topic: 2 
Words: 0.071*"ulica" + 0.045*"zielony" + 0.038*"park" + 0.035*"pieszy" + 0.027*"budowo" + 0.026*"do" + 0.020*"sportowy" + 0.018*"osiedle" + 0.016*"dorosły" + 0.013*"warszawa"
Topic: 3 
Words: 0.062*"ul" + 0.026*"numer" + 0.026*"rowerowy" + 0.025*"parkingowy" + 0.022*"zabawić" + 0.016*"remont" + 0.015*"droga" + 0.014*"biblioteka" + 0.012*"modernizacja" + 0.011*"gra"
Topic: 4 
Words: 0.027*"dziecko" + 0.020*"miejski" + 0.019*"chodnik" + 0.016*"nowa" + 0.015*"bezpieczny" + 0.015*"centrum" + 0.012*"boisko" + 0.010*"publiczny" + 0.009*"rekreacyjny" + 0.008*"ruch"


In [157]:
# Wizualizacja tematów
vis_data = vis.prepare(lda_model, corpus_tfidf, vocab)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis_data)

## 2019

In [158]:
df_2019 = pd.read_csv('/content/drive/My Drive/Konkursy Kaggle/Budżet obywatelski/Morfeusz_2019.csv')
df_2019.head()

,rok,miasto,nazwa,start_ind,end_ind,form,lemma,tag,qualifiers,_
0,2019,GDYNIA,"Parki kieszonkowe W 9 dzielnicach: Dqbrowa, Grabówek, Wzgórze Swietego Maksymiliana, Chylonia, Pogórze, Dziatki Lesne, Leszczynki, Pustki Cisowskie, Wielki Kack",0,1,Parki,park,subst:pl:nom.acc.voc:m3,['nazwa_pospolita'],[]
1,2019,GDYNIA,"Parki kieszonkowe W 9 dzielnicach: Dqbrowa, Grabówek, Wzgórze Swietego Maksymiliana, Chylonia, Pogórze, Dziatki Lesne, Leszczynki, Pustki Cisowskie, Wielki Kack",0,1,Parki,parka,subst:sg:gen:f,['nazwa_pospolita'],[]
2,2019,GDYNIA,"Parki kieszonkowe W 9 dzielnicach: Dqbrowa, Grabówek, Wzgórze Swietego Maksymiliana, Chylonia, Pogórze, Dziatki Lesne, Leszczynki, Pustki Cisowskie, Wielki Kack",0,1,Parki,parka,subst:pl:nom.acc.voc:f,['nazwa_pospolita'],[]
3,2019,GDYNIA,"Parki kieszonkowe W 9 dzielnicach: Dqbrowa, Grabówek, Wzgórze Swietego Maksymiliana, Chylonia, Pogórze, Dziatki Lesne, Leszczynki, Pustki Cisowskie, Wielki Kack",0,1,Parki,Parka:Sf,subst:sg:gen:f,"['imię', 'nazwisko']",[]
4,2019,GDYNIA,"Parki kieszonkowe W 9 dzielnicach: Dqbrowa, Grabówek, Wzgórze Swietego Maksymiliana, Chylonia, Pogórze, Dziatki Lesne, Leszczynki, Pustki Cisowskie, Wielki Kack",0,1,Parki,Parka:Sf,subst:pl:nom.acc.voc:f,"['imię', 'nazwisko']",[]


In [159]:
df_2019['lemma_clean'] = df_2019['lemma'].str.split(':', expand=True)[0]
df_2019['tokens'] = df_2019['lemma_clean'].apply(simple_preprocess)
df_2019_ = df_2019[df_2019["tokens"].str.len() != 0]

In [160]:
#tfidf
processed_docs = df_2019_['tokens']
vocab = gensim.corpora.Dictionary(processed_docs)
corpus = [vocab.doc2bow(doc) for doc in processed_docs]
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

lda_model = gensim.models.LdaMulticore(corpus_tfidf, num_topics=5, id2word=vocab, passes=2, workers=10)

for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.027*"pieszy" + 0.023*"szkoła" + 0.018*"rowerowy" + 0.016*"dzielnica" + 0.016*"sportowy" + 0.010*"dorosły" + 0.008*"oda" + 0.008*"stare" + 0.008*"zakup" + 0.007*"plenerowy"
Topic: 1 
Words: 0.054*"dla" + 0.053*"do" + 0.051*"nowy" + 0.030*"mini" + 0.028*"budowo" + 0.021*"bezpieczny" + 0.019*"zabawić" + 0.018*"podstawowy" + 0.016*"nowa" + 0.015*"boisko"
Topic: 2 
Words: 0.054*"ul" + 0.044*"park" + 0.041*"plac" + 0.031*"dziecko" + 0.026*"zabawa" + 0.023*"miejski" + 0.019*"numer" + 0.016*"nr" + 0.014*"zieleń" + 0.012*"sp"
Topic: 3 
Words: 0.220*"na" + 0.053*"zielony" + 0.042*"przy" + 0.021*"chodnik" + 0.019*"nawierzchnia" + 0.018*"budowa" + 0.015*"remont" + 0.015*"remonta" + 0.014*"parkingowy" + 0.007*"nie"
Topic: 4 
Words: 0.077*"ulica" + 0.021*"osiedle" + 0.021*"oraz" + 0.017*"stary" + 0.014*"teren" + 0.012*"zielone" + 0.011*"drogi" + 0.011*"mały" + 0.011*"parking" + 0.010*"łódź"


In [161]:
tags = ['prep', 'conj', 'interj', 'part']
df_2019_clean = df_2019_[~df_2019_['tag'].str.contains('|'.join(tags))]
df_2019_clean.sample(10)

,rok,miasto,nazwa,start_ind,end_ind,form,lemma,tag,qualifiers,_,lemma_clean,tokens
32801,2019,GLIWICE,"Organizacja zajec fitness i aerobik dla 50 kobiet Z osiedla, raz W tygodniu Z przerwa wakacyjna.",16,17,wakacyjna,wakacyjny,adj:sg:nom.voc:f:pos,[],[],wakacyjny,[wakacyjny]
54995,2019,KIELCE,Remont nawierzchni drogi osiedlowej od ul. Piłsudskiego do ul. Orląt Lwowskich,12,13,Lwowskich,Lwowska,subst:pl:loc:f,['nazwisko'],[],Lwowska,[lwowska]
98433,2019,ŁÓDŹ,Kolorowy plac zabaw w PM 43.,0,1,Kolorowy,kolorowy:S,subst:sg:voc:m1,['nazwa_pospolita'],[],kolorowy,[kolorowy]
2761,2019,GDYNIA,"Cykliczne potancówki przy muzyce polskich kapel ludowych, szczególnie kapel Z Pomorza i okolic (kaszubskie, kociewskie, Z Zutaw itp.) oraz pokazy filmów",21,22,itp,i_tym_podobne,brev:pun,[],[],i_tym_podobne,[i_tym_podobne]
62395,2019,KALISZ,Modernizacja bazy sportowej przy Szkole Podstawowej Nr 4 im. Marii Dąbrowskiej.,2,3,sportowej,sportowy,adj:sg:dat:f:pos,[],[],sportowy,[sportowy]
158069,2019,WARSZAWA,Nowe kosze na śmieci na ulicy Kowalczyka i Krzyżówki,3,4,śmieci,śmieć:Sm1.m2,subst:pl:gen:m2,['nazwa_pospolita'],['pogard.'],śmieć,[śmieć]
102087,2019,ŁÓDŹ,Bezpieczna droga do działek w Łagiewnikach.,1,2,droga,drogi:A,adj:sg:nom.voc:f:pos,[],[],drogi,[drogi]
59398,2019,OLSZTYN,162. REWITALIZACJA WYBIEGU DLA PSÓW PRZY UL. WESTERPLATTE,3,4,WYBIEGU,wybieg,subst:sg:voc:m3,['nazwa_pospolita'],[],wybieg,[wybieg]
95537,2019,GORZÓW WLKP.,Parking na 16 miejsc postojowych przy ul. Gwiaździstej,4,5,postojowych,postojowe,subst:pl:gen:n:ncol,['nazwa_pospolita'],[],postojowe,[postojowe]
143690,2019,RZESZÓW,Bajka dla dzieci przedstawiona innowacyjną sztuką piaskowych animacji,0,1,Bajka,Bajek:Sm1,subst:sg:gen.acc:m1,['nazwisko'],[],Bajek,[bajek]


In [162]:
#tfidf
processed_docs = df_2019_clean['tokens']
vocab = gensim.corpora.Dictionary(processed_docs)
corpus = [vocab.doc2bow(doc) for doc in processed_docs]
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

lda_model = gensim.models.LdaMulticore(corpus_tfidf, num_topics=5, id2word=vocab, passes=2, workers=10)

for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.045*"nowy" + 0.019*"rowerowy" + 0.018*"chodnik" + 0.013*"centrum" + 0.012*"oraz" + 0.011*"drogi" + 0.011*"dorosły" + 0.009*"wraz" + 0.008*"zakupić" + 0.007*"sala"
Topic: 1 
Words: 0.038*"dziecko" + 0.031*"mini" + 0.025*"miejski" + 0.021*"sportowy" + 0.018*"nowa" + 0.017*"zieleń" + 0.016*"parkingowy" + 0.015*"remont" + 0.012*"mały" + 0.011*"oda"
Topic: 2 
Words: 0.086*"ulica" + 0.063*"ul" + 0.050*"plac" + 0.027*"szkoła" + 0.023*"bezpieczny" + 0.021*"podstawowy" + 0.017*"remonta" + 0.014*"miejsce" + 0.014*"biblioteka" + 0.012*"zajęcie"
Topic: 3 
Words: 0.047*"park" + 0.025*"zabawa" + 0.022*"osiedle" + 0.020*"stary" + 0.020*"nr" + 0.020*"numer" + 0.019*"dzielnica" + 0.017*"zabawić" + 0.016*"nawierzchnia" + 0.016*"boisko"
Topic: 4 
Words: 0.055*"zielony" + 0.030*"pieszy" + 0.030*"budowo" + 0.030*"do" + 0.020*"budowa" + 0.018*"młodzież" + 0.017*"teren" + 0.014*"on" + 0.012*"zająć" + 0.012*"zielone"


In [163]:
# Wizualizacja tematów
vis_data = vis.prepare(lda_model, corpus_tfidf, vocab)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis_data)

## 2020

In [164]:
df_2020= pd.read_csv('/content/drive/My Drive/Konkursy Kaggle/Budżet obywatelski/Morfeusz_2020.csv')
df_2020.head()

,rok,miasto,nazwa,start_ind,end_ind,form,lemma,tag,qualifiers,_
0,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",0,1,Doposażenie,doposażyć,ger:sg:nom.acc:n:perf:aff,[],[]
1,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",1,2,parku,park,subst:sg:gen:m3,['nazwa_pospolita'],[]
2,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",1,2,parku,park,subst:sg:loc:m3,['nazwa_pospolita'],[]
3,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",1,2,parku,park,subst:sg:voc:m3,['nazwa_pospolita'],[]
4,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",2,3,pomiędzy,pomiędzy,prep:acc,[],[]


In [165]:
df_2020['lemma_clean'] = df_2020['lemma'].str.split(':', expand=True)[0]
df_2020['tokens'] = df_2020['lemma_clean'].apply(simple_preprocess)
df_2020_ = df_2020[df_2020["tokens"].str.len() != 0]

In [166]:
#tfidf
processed_docs = df_2020_['tokens']
vocab = gensim.corpora.Dictionary(processed_docs)
corpus = [vocab.doc2bow(doc) for doc in processed_docs]
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

lda_model = gensim.models.LdaMulticore(corpus_tfidf, num_topics=5, id2word=vocab, passes=2, workers=10)

for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.080*"zielony" + 0.075*"ulica" + 0.041*"przy" + 0.016*"stary" + 0.016*"podstawowy" + 0.016*"numer" + 0.014*"nawierzchnia" + 0.014*"remont" + 0.010*"po" + 0.010*"droga"
Topic: 1 
Words: 0.224*"na" + 0.043*"park" + 0.023*"dziecko" + 0.023*"budowo" + 0.016*"teren" + 0.013*"boisko" + 0.012*"biblioteka" + 0.011*"drogi" + 0.009*"zajęcie" + 0.009*"zielona"
Topic: 2 
Words: 0.054*"dla" + 0.038*"plac" + 0.032*"łódź" + 0.027*"szkoła" + 0.024*"zabawa" + 0.022*"dzielnica" + 0.021*"chodnik" + 0.019*"rowerowy" + 0.016*"zabawić" + 0.010*"parkingowy"
Topic: 3 
Words: 0.047*"nowy" + 0.030*"pieszy" + 0.027*"mini" + 0.026*"miejski" + 0.021*"bezpieczny" + 0.017*"zieleń" + 0.016*"zielone" + 0.015*"nr" + 0.015*"sportowy" + 0.014*"budowa"
Topic: 4 
Words: 0.055*"ul" + 0.049*"do" + 0.022*"nowa" + 0.019*"osiedle" + 0.018*"oraz" + 0.014*"młodzież" + 0.012*"sp" + 0.011*"mały" + 0.010*"zająć" + 0.010*"parking"


In [167]:
tags = ['prep', 'conj', 'interj', 'part']
df_2020_clean = df_2020_[~df_2020_['tag'].str.contains('|'.join(tags))]
df_2020_clean.sample(10)

,rok,miasto,nazwa,start_ind,end_ind,form,lemma,tag,qualifiers,_,lemma_clean,tokens
37193,2020,KATOWICE,Sokolska! Ratunku! Chodnik!,0,1,Sokolska,Sokolska,subst:sg:nom:f,['nazwisko'],[],Sokolska,[sokolska]
22267,2020,GDAŃSK,Tablice informacyjno-ogłoszeniowe dla rowerzystów,3,4,ogłoszeniowe,ogłoszeniowy,adj:sg:acc:n:pos,[],[],ogłoszeniowy,[ogłoszeniowy]
122890,2020,ŁÓDŹ,"Unihokej bawi, uczy i wychowuje - ogólnodostępne zajęcia dla dzieci i młodzieży - Hala Anilana, ul. Sobolowa",8,9,zajęcia,zająć,ger:pl:nom.acc:n:perf:aff,[],[],zająć,[zająć]
78473,2020,POZNAŃ,Budowa lekkiej hali sportowej przy SP20,1,2,lekkiej,lekki,adj:sg:loc:f:pos,[],[],lekki,[lekki]
60001,2020,SOSNOWIEC,Rewitalizacja przestrzeni miejskiej W rejonie ul. Jana Dtugosza.,4,5,rejonie,rejon,subst:sg:voc:m3,['nazwa_pospolita'],[],rejon,[rejon]
138283,2020,SIEDLCE,Budowa chodnika na ulicy Grabowej (po stronie zachodniej) od ulicy Starowisjskiej do ulicy Klonowej.,1,2,chodnika,chodnik,subst:sg:gen:m3,['nazwa_pospolita'],[],chodnik,[chodnik]
71762,2020,KALISZ,Kaliski Ogród z Pasieką na terenie UAM przy ul. Nowy Świat.,0,1,Kaliski,kaliski,adj:sg:acc:m3:pos,[],[],kaliski,[kaliski]
95888,2020,BYDGOSZCZ,Nowe nasadzenia na terenie stadionu miejskiego TKKF (dawniej Budowlani),0,1,Nowe,Nowe,subst:sg:nom.acc.voc:n:ncol,['nazwa_geograficzna'],[],Nowe,[nowe]
129360,2020,ŁÓDŹ,NOWE KSIĄŻKI DLA FILII NR 68 DLA DZIECI I MŁODZIEŻY BM.,0,1,NOWE,nowy:A,adj:pl:acc:m2.m3.f.n:pos,[],[],nowy,[nowy]
63348,2020,TYCHY,Organizacja wydarzeń sportowych z okazji 50-lecia\nGKS Tychy,0,1,Organizacja,organizacja,subst:sg:nom:f,['nazwa_pospolita'],[],organizacja,[organizacja]


In [168]:
#tfidf
processed_docs = df_2020_clean['tokens']
vocab = gensim.corpora.Dictionary(processed_docs)
corpus = [vocab.doc2bow(doc) for doc in processed_docs]
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

lda_model = gensim.models.LdaMulticore(corpus_tfidf, num_topics=5, id2word=vocab, passes=2, workers=10)

for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.042*"plac" + 0.027*"budowo" + 0.024*"bezpieczny" + 0.019*"sportowy" + 0.018*"zielone" + 0.017*"stary" + 0.015*"biblioteka" + 0.015*"boisko" + 0.013*"dorosły" + 0.012*"zajęcie"
Topic: 1 
Words: 0.083*"zielony" + 0.063*"ul" + 0.047*"park" + 0.036*"pieszy" + 0.031*"miejski" + 0.020*"dzielnica" + 0.020*"rowerowy" + 0.019*"nowa" + 0.017*"budowa" + 0.017*"młodzież"
Topic: 2 
Words: 0.084*"ulica" + 0.052*"nowy" + 0.040*"łódź" + 0.031*"dziecko" + 0.028*"do" + 0.028*"zabawa" + 0.024*"chodnik" + 0.019*"teren" + 0.013*"zająć" + 0.012*"droga"
Topic: 3 
Words: 0.029*"mini" + 0.028*"szkoła" + 0.018*"osiedle" + 0.017*"nr" + 0.014*"on" + 0.012*"wola" + 0.010*"oraz" + 0.009*"nowość" + 0.009*"poprawa" + 0.008*"pies"
Topic: 4 
Words: 0.018*"podstawowy" + 0.018*"zieleń" + 0.016*"zabawić" + 0.015*"numer" + 0.015*"centrum" + 0.014*"nawierzchnia" + 0.013*"remont" + 0.012*"miejsce" + 0.011*"sp" + 0.011*"drogi"


In [169]:
# Wizualizacja tematów
vis_data = vis.prepare(lda_model, corpus_tfidf, vocab)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis_data)

In [170]:
stopwords = ['do', 'wraz', 'oraz', 'ul', 'ulica', 'nr', 'numer', 'to', 'on', 'przy', 'dla', 'na', 'po','jak', 'ten', 'nie', 'od', 'oda']
df_2020_clean4 = df_2020_[~df_2020_['lemma'].str.contains('|'.join(stopwords))]
df_2020_clean4.sample(10)

,rok,miasto,nazwa,start_ind,end_ind,form,lemma,tag,qualifiers,_,lemma_clean,tokens
76999,2020,POZNAŃ,Festiwal Jana Konstantego Żupańskiego 2021,3,4,Żupańskiego,żupański,adj:sg:gen:m1.m2.m3.n:pos,[],[],żupański,[żupański]
5648,2020,GDYNIA,"Łąki kwietne w Redłowie - estetyzacja podwórek w obrębie ulic Powstania Wielkopolskiego, Powstania Śląskiego",8,9,obrębie,obrąb,subst:sg:loc:m3,['nazwa_pospolita'],[],obrąb,[obrąb]
140608,2020,OPOLE,Bo w OPO. zrealizujemy ten projekt koncertowo!,4,5,zrealizujemy,zrealizować,fin:pl:pri:perf,[],[],zrealizować,[zrealizować]
121805,2020,ŁÓDŹ,Łódź dla ptaków - czyli działamy na rzecz ptasiej społeczności (Mileszki).,4,5,czyli,czyli:C,conj,[],[],czyli,[czyli]
80016,2020,KOSZALIN,3. Osiedlowy zawrót głowy - czyli miejsce dające trochę radości naszej ulicy i okolicy.,8,9,dające,dawać,pact:sg:nom.acc.voc:n:imperf:aff,[],[],dawać,[dawać]
176302,2020,KRAKÓW,Winda dla chorych W Szpitalu im. St. Zeromskiego - pomozesz?,4,5,Szpitalu,szpital,subst:sg:loc:m3,['nazwa_pospolita'],[],szpital,[szpital]
136021,2020,NOWY SĄCZ,Nowy Sącz - miasto Kobiet - Bądź dla siebie dobra - warsztaty rozwojowe dla kobiet,0,1,Nowy,Nowy:Sm1,subst:sg:voc:m1,['nazwisko'],[],Nowy,[nowy]
30677,2020,CZĘSTOCHOWA,CZESTOCHOWIANIE STERYLIZUJA I KASTRUJA SWOJE PSY I KOTY ZA DARMO,7,8,KOTY,kota,subst:sg:gen:f,['nazwa_pospolita'],[],kota,[kota]
41665,2020,KATOWICE,Zakup uzupełniający sprzętu ratowniczo-gaśniczego dla Ochotniczej Straży Pożarnej w Katowicach Zarzeczu,1,2,uzupełniający,uzupełniać,pact:sg:acc:m3:imperf:aff,[],[],uzupełniać,[uzupełniać]
87252,2020,WROCŁAW,OGROMNE ROSARIUM - „FLAMING WŚRÓD RÓŻ” = Długie Tunele Różane + Klomby Róż + Plac zabaw „MIESZKAŃCY OGRODU”,6,7,RÓŻ,Róż:Sm1,subst:sg:nom:m1,['nazwisko'],[],Róż,[róż]


In [171]:
#tfidf
processed_docs = df_2020_clean4['tokens']
vocab = gensim.corpora.Dictionary(processed_docs)
corpus = [vocab.doc2bow(doc) for doc in processed_docs]
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

lda_model = gensim.models.LdaMulticore(corpus_tfidf, num_topics=7, id2word=vocab, passes=2, workers=10)

for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.068*"pieszy" + 0.043*"bezpieczny" + 0.038*"nowa" + 0.021*"droga" + 0.018*"parkingowy" + 0.017*"oświetlić" + 0.016*"wolny" + 0.015*"sala" + 0.015*"skwer" + 0.010*"rekreacja"
Topic: 1 
Words: 0.081*"park" + 0.036*"rowerowy" + 0.024*"sp" + 0.017*"pies" + 0.016*"warsztat" + 0.016*"książka" + 0.012*"czyli" + 0.012*"aktywny" + 0.010*"poprawa" + 0.010*"zielenić"
Topic: 2 
Words: 0.092*"nowy" + 0.040*"dzielnica" + 0.031*"stary" + 0.030*"zabawić" + 0.023*"miejsce" + 0.021*"drogi" + 0.021*"zajęcie" + 0.019*"parking" + 0.017*"wzdłuż" + 0.016*"rekreacyjny"
Topic: 3 
Words: 0.058*"szkoła" + 0.057*"dziecko" + 0.018*"nowość" + 0.018*"ścieżka" + 0.016*"plenerowy" + 0.015*"filia" + 0.015*"edukacyjny" + 0.014*"przejść" + 0.013*"ii" + 0.013*"zajęcia"
Topic: 4 
Words: 0.072*"plac" + 0.057*"mini" + 0.035*"zieleń" + 0.027*"mały" + 0.018*"zakup" + 0.016*"drzewo" + 0.015*"budynek" + 0.012*"rok" + 0.011*"osiedlowy" + 0.011*"bezpieczeństwo"
Topic: 5 
Words: 0.064*"łódź" + 0.036*"osiedle" + 0.